# Dot Product (Neural Networks)

In [2]:
import numba 
from numba import cuda, jit, njit
import numpy as np
from functools import reduce
import datetime as dt

def now():
    return dt.datetime.now()

with open('offline_1000000.csv', 'r', encoding='utf-8') as arq:
    array = np.array([int(i) for i in arq.read().split('\n') if i])
ref = max(array)
array.size, array.dtype, ref

(1000000, dtype('int64'), 10013464795)

In [5]:
inputs = [1, 2, 3, 2.5]
weights = [[0.2, 0.8, -0.5, 1.0], [0.5, -0.91, 0.26, -0.5], [ -0.26, -0.27, 0.17, 0.87]]
biases = [2, 3, 0.5]

layer_outputs = []
for neuron_weigths, neuron_bias in zip(weights, biases):
    neuron_output = 0
    for n_input, weight in zip(inputs, neuron_weigths):
        neuron_output += n_input * weight
    neuron_output += neuron_bias
    layer_outputs.append(neuron_output)

In [6]:
layer_outputs

[4.8, 1.21, 2.385]

In [10]:
def dot(weights, inputs, bias):
    return np.dot(inputs, weights) + bias

inputs = np.array([1, 2, 3, 2.5])
weights = [0.2, 0.8, -0.5, 1.0]
bias = 2.0
dot(weights, inputs, bias)

4.8

In [19]:
inputs = np.array([1, 2, 3, 2.5], np.float32)
weights = np.array([0.2, 0.8, -0.5, 1.0], dtype=np.float32)
bias = np.float32(2.0)
ref = np.dot(inputs, weights) + bias
ref

4.8

In [111]:
@njit()
def dot(weights, inputs, bias):
    return np.dot(inputs, weights) + bias
%timeit dot(weights, inputs, bias)
f'{round(np.average([0.00000101, 0.00000101, 0.00000108, 0.00000103, 0.00000101, 0.00000106]), 8):.8f}', ref == dot(weights, inputs, bias)

1.22 µs ± 22.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


('0.00000103', True)

In [122]:
def dot(weights, inputs, bias):
    return np.dot(inputs, weights) + bias
%timeit dot(weights, inputs, bias)
f'{round(np.average([0.00000275, 0.00000263, 0.00000267, 0.00000273, 0.00000274, 0.00000347, 0.00000323]), 8):.8f}', ref == dot(weights, inputs, bias)

3.23 µs ± 46.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


('0.00000283', True)

In [112]:
@jit()
def dot(weights, inputs, bias):
    return np.dot(inputs, weights) + bias
%timeit dot(weights, inputs, bias)
f'{round(np.average([0.00000101, 0.00000113, 0.00000104, 0.000000984, 0.00000988]), 8):.8f}', ref == dot(weights, inputs, bias)

1.22 µs ± 33.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


('0.00000281', True)

In [120]:
@njit
def dot(weights, inputs, bias):
    return np.sum(np.multiply(inputs, weights)) + bias
%timeit dot(weights, inputs, bias)
f'{round(np.average([0.00000397, 0.00000301, 0.00000356, 0.00000547, 0.00000386, 0.00000326, 0.00000557, 0.00000319]), 8):.8f}, {ref == dot(weights, inputs, bias)}'

The slowest run took 7.80 times longer than the fastest. This could mean that an intermediate result is being cached.
3.19 µs ± 3.48 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


'0.00000410, True'

In [117]:
def dot(weights, inputs, bias):
    return np.sum(np.multiply(inputs, weights)) + bias
%timeit dot(weights, inputs, bias)
f'{round(np.average([0.0000012, 0.00000125, 0.0000117, 0.0000012, 0.0000118, 0.0000113]), 8):.8f}, {ref == dot(weights, inputs, bias)}'

11.3 µs ± 193 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


'0.00000543, True'

In [110]:
@jit
def dot(weights, inputs, bias):
    return np.sum(np.multiply(inputs, weights)) + bias
%timeit dot(weights, inputs, bias)
f'{round(np.average([0.00000416, 0.0000041, 0.00000339, 0.00000599, 0.00000364, 0.0000042]), 8):.8f}, {ref == dot(weights, inputs, bias)}'

The slowest run took 13.14 times longer than the fastest. This could mean that an intermediate result is being cached.
4.2 µs ± 5.8 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


'0.00000426, True'

In [113]:
@njit(parallel=True)
def dot(weights, inputs, bias):
    return np.dot(inputs, weights) + bias
%timeit dot(weights, inputs, bias)
f'{round(np.average([0.0000236, 0.0000276, 0.0000307, 0.0000283, 0.0000276]), 8):.8f}, {ref == dot(weights, inputs, bias)}'

The slowest run took 39.00 times longer than the fastest. This could mean that an intermediate result is being cached.
109 µs ± 207 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


'0.00002756, True'